### Getting the files & import statements ###

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
# Get participant number 
participant_number = input("Enter participant number: ")

base_path = "/Users/alina/Downloads/Thesis/raw/"

# Folder path for the participant
folder_path = os.path.join(base_path, f"Participant {participant_number}")

# Get recording number 
recording_number = input("Enter recording number: ")

# Recording folder path
recording_path = os.path.join(folder_path, f"P{participant_number}_{recording_number}")

# Get files
world_timestamps_file = os.path.join(recording_path, "world_timestamps.csv")
detections_file = os.path.join(recording_path, f"detections_{participant_number}_{recording_number}.csv")
fixations_file = os.path.join(recording_path, "fixations.csv")
face_file = os.path.join(recording_path, "fixations_on_face.csv")

# Load as dfs
world_timestamps_df = pd.read_csv(world_timestamps_file) #contains timestamps for each video frame
detections_df = pd.read_csv(detections_file) #contains detections from YOLO
fixations_df = pd.read_csv(fixations_file) #fixations data
face_df = pd.read_csv(face_file) #fixations on face data from FaceMapper

### Matching video frames and timestamps

In [4]:
## function to see if the gaze is inside the bounding box of an object
def is_gaze_in_bbox(gaze_x, gaze_y, x_min, y_min, x_max, y_max):
    return x_min <= gaze_x <= x_max and y_min <= gaze_y <= y_max

## function to prioritise the smallest object fixated
def filter_objects_by_area(group):
    group = group.copy()
    group["area"] = (group["X_max"] - group["X_min"]) * (group["Y_max"] - group["Y_min"])

    # Find the minimum area
    min_area = group["area"].min()
    
    # Get all objects with the minimum area
    smallest_objects = group[group["area"] == min_area]
    
    if len(smallest_objects) == 1:
        return smallest_objects
    else:
        person_objects = smallest_objects[smallest_objects["Class"] == "person"]
        if not person_objects.empty:
            # Return only one person object (the first one)
            return person_objects.iloc[[0]]

    # If no person among smallest, return just one of the smallest objects
    return smallest_objects.iloc[[0]]

In [5]:
## world_timestamps.csv file from pupil cloud represents timestamps for each video frame
# merge_asof is like a nearest match join for ordered time-series data

# renaming so all timestamp columns are named the same to merge the dfs
fixations_df["timestamp [ns]"] = fixations_df["start timestamp [ns]"]

## making a video_df which contains matching frames and ts
ts = world_timestamps_df['timestamp [ns]']

video_df = pd.DataFrame({
        "Frame": np.arange(len(world_timestamps_df)),
        "timestamp [ns]": ts,
    })

# Make sure everything is sorted by timestamp so merge_asof works properly
fixations_df = fixations_df.sort_values("start timestamp [ns]")
world_timestamps_df = world_timestamps_df.sort_values("timestamp [ns]")

# Merge-asof to assign the closest frame_id to each fixation
fixations_with_frame = pd.merge_asof(
    fixations_df,
    video_df,
    on="timestamp [ns]",
    direction="nearest",  # to match the closest timestamp from video_df, whether it's before or after the fixation timestamp
) 

# merging to assign fixations to frames
fixations_with_detections = pd.merge(
    fixations_with_frame,
    detections_df,
    on="Frame",
    how="left" # keep all the fixations, even if a frame has no detection => nan
)

# calculate in_bbox 
fixations_with_detections["in_bbox"] = fixations_with_detections.apply(
    lambda row: is_gaze_in_bbox(
        row["fixation x [px]"], row["fixation y [px]"],
        row["X_min"], row["Y_min"], row["X_max"], row["Y_max"]
    ),
    axis=1
)

# filter only fixations that match at least one bounding box
in_bbox_fixations = fixations_with_detections[fixations_with_detections["in_bbox"] == True]

#  group by fixation ID and drop "person" if there is another class in same fixation group
filtered_fixations = in_bbox_fixations.groupby(
    "Frame", group_keys=False # grouping on frame as i need fixation id column later
).apply(filter_objects_by_area, include_groups=False)

fixations_on_persons = filtered_fixations.loc[filtered_fixations["Class"] == "person"]
fixations_on_persons.head()

,section id,recording id,fixation id,start timestamp [ns],end timestamp [ns],duration [ms],fixation x [px],fixation y [px],azimuth [deg],elevation [deg],timestamp [ns],X_min,Y_min,X_max,Y_max,Confidence,Class,in_bbox,area
101,2ec448bc-1c60-4549-9c03-6b327b7f7e37,8ef60243-2690-4dac-9837-d7b0a844badb,9,1742392587915153145,1742392588095278145,180,923.603,270.347,6.874680,21.503199,1742392587915153145,905.394897,229.592819,942.507629,324.734344,0.390972,person,True,3530.961923
117,2ec448bc-1c60-4549-9c03-6b327b7f7e37,8ef60243-2690-4dac-9837-d7b0a844badb,11,1742392588390535145,1742392588545775145,155,340.903,317.412,-31.795667,17.612949,1742392588390535145,320.437744,249.708374,370.772827,365.454376,0.715674,person,True,5826.084628
128,2ec448bc-1c60-4549-9c03-6b327b7f7e37,8ef60243-2690-4dac-9837-d7b0a844badb,12,1742392588585775145,1742392589331397145,746,356.500,248.049,-31.361388,21.889288,1742392588585775145,330.336975,244.071960,378.671814,362.776459,0.608458,person,True,5737.562798
175,2ec448bc-1c60-4549-9c03-6b327b7f7e37,8ef60243-2690-4dac-9837-d7b0a844badb,16,1742392590327393145,1742392590477517145,150,1278.216,267.387,30.670970,20.753340,1742392590327393145,1229.278320,236.690857,1294.873657,357.916016,0.355547,person,True,7951.805127
178,2ec448bc-1c60-4549-9c03-6b327b7f7e37,8ef60243-2690-4dac-9837-d7b0a844badb,17,1742392590497517145,1742392590998029145,501,1311.275,263.910,32.933544,20.827633,1742392590497517145,1242.247070,237.412338,1599.308105,1197.999390,0.840304,person,True,342988.206928


### Creating a new file with categorised fixations

In [6]:
# filtering to only fixations on bodies
fixations_on_persons = fixations_on_persons[fixations_on_persons["in_bbox"] == True]

# initialising a df for all types of fixations
categorized_fixations_all = []

# using FaceMapper file to get all the basic fixations information
for _, row in face_df.iterrows():
    timestamp = row['start timestamp [ns]']
    fixation_id = row['fixation id']
    fix_duration = (row['end timestamp [ns]'] - row['start timestamp [ns]'])/1000000     #converting to ms
    fixation_type = "background" #make default type

    # face fixations
    if row.get('fixation on face') == True:
        fixation_type = "face"

    # body fixations (only if not already labeled as face)
    elif not fixations_on_persons[fixations_on_persons['timestamp [ns]'] == timestamp].empty:
        fixation_type = "body"
        matching_row = fixations_on_persons[fixations_on_persons['timestamp [ns]'] == timestamp].iloc[0]
        fixation_id = matching_row['fixation id']
        fix_duration = matching_row['duration [ms]']

    categorized_fixations_all.append({
        'timestamp [ns]': timestamp,
        'type': fixation_type,
        'fixation id': fixation_id,
        'duration [ms]': fix_duration
    })

# saving a file with fixations categorised by face, body and background
all_categorized_fixations_df = pd.DataFrame(categorized_fixations_all)
saving_path = os.path.join(recording_path, f"fixations_on_everything_{participant_number}_{recording_number}.csv")
all_categorized_fixations_df.to_csv(saving_path, index=False)

all_categorized_fixations_df.head()

,timestamp [ns],type,fixation id,duration [ms]
0,1742392581649306145,background,1,1986.866
1,1742392583781297145,background,2,1596.493
2,1742392585417789145,background,3,450.498
3,1742392585903287145,background,4,710.622
4,1742392586664034145,background,5,405.373
